Arquivo disponível em https://www.kaggle.com/s1m0n38/clash-royale-matches-dataset.

Neste Notebook eu tento mostrar alguns Decks bons do Clash Royale, extraídos dos melhores Players e Clans deste conjunto de dados.

Clash Royale é um jogo de celular muito popular disponível nas lojas de aplicativos das principais plataformas.

De início importamos algumas bibliotecas e o arquivo de dados.
O conjunto de dados é muito longo (mais de 700 mil). Nós vamos trabalhar com 20.000 linhas por enquanto. Isso vai levar algum tempo para correr, mas pandas e jupyter podem fazer isso.

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
with open('/Users/rafaelferri/Arquivos/Clash/matches.txt') as file:
    CR = [ x.strip() for x in file.readlines()]

cr_normal = [json_normalize(eval(r1))for r1 in CR[0:20000]]
CRM = pd.concat(cr_normal, ignore_index = True)
CRM.columns = ['Left Clan','Left Deck','Left Player','Left Trophy','Right Clan','Right Deck','Right Player','Right Trophy','Result','Time','Type']

Vamos abrir algumas informações importantes sobre coroas ganhas, coroas perdidas e resultado da partida. E, para o resultado, eu coloquei um valor de 1 para uma vitória, -1 para a derrota e 0 para empates.

Em seguida está uma grande mudança no arquivo: eu divido os dados em dois DataFrames (dados dos jogadores à esquerda e dados dos jogadores da direita) e concateno os dois em um só. Assim, o nosso arquivo de 20.000 linhas dobra e vai para 40.000 linhas. Isso vai facilitar quando nós agruparmos algumas informações de Decks, Clans, etc.

In [3]:
CRM['Left Crowns Won'] = [int(stars[0]) for stars in CRM['Result']]
CRM['Right Crowns Won'] = [int(stars[1]) for stars in CRM['Result']]

CRM['Left Result'] = [int(1) if(left>right) else int(-1) if(left<right) else int(0) for left, right in zip(CRM['Left Crowns Won'], CRM['Right Crowns Won'])]
CRM['Right Result'] = [int(-1) if(left>right) else int(1) if(left<right) else int(0) for left, right in zip(CRM['Left Crowns Won'], CRM['Right Crowns Won'])]
CRM.drop('Result', axis=1, inplace=True)

CRMl = CRM[['Left Clan','Left Deck','Left Player','Left Trophy','Left Crowns Won','Right Crowns Won','Left Result','Time','Type']]
CRMl.columns = ['Clan','Deck','Player','Trophy','Crowns Won','Crowns Lost','Result','Time','Type']

CRMr = CRM[['Right Clan','Right Deck','Right Player','Right Trophy','Right Crowns Won','Left Crowns Won','Right Result','Time','Type']]
CRMr.columns = ['Clan','Deck','Player','Trophy','Crowns Won','Crowns Lost','Result','Time','Type']

CRMf = pd.concat([CRMl, CRMr])
CRMf.index = range(len(CRMf))

In [4]:
# Abrindo as informações dos Decks
Army_colNames = np.hstack([["Troop "+str(i+1) for i in range(8)], ["Level "+str(i+1) for i in range(8)]])
Army = pd.DataFrame(data=[np.hstack([[army[0] for army in x], [int(army[1]) for army in x]]) for x in CRMf['Deck']], columns=Army_colNames)

CRMf = pd.concat([CRMf, Army], axis=1, join='inner')
CRMf.drop(['Deck'], axis=1, inplace=True)

Vamos começar selecionando os Clans com 10 vitórias ou mais e dividindo este número com a contagem deste Clan. Isso nos retornará uma porcentagem de vitória de cada clã. Finalmente, classificamos esse resultado por ordem decrescente.

In [5]:
# Melhores Clãs

ClasVitorias = CRMf.groupby('Clan')['Result'].sum().sort_values(ascending=False)
ClasVitorias10 = ClasVitorias[ClasVitorias > 10]

ClasContagem = CRMf.groupby('Clan')['Clan'].count().sort_values(ascending=False)

PercVitoriaCla = (ClasVitorias10 / ClasContagem)*100
PercVitoriaCla = pd.DataFrame(PercVitoriaCla.sort_values(ascending=False), columns=['Perc.'])

Agora vamos melhorar os resultados do Melhor Clã, incluindo a quantidade de troféus que eles têm. Para isso, vamos encontrar a média de troféus de cada Clã e depois multiplicar isso pelo número de jogadores únicos de cada Clã. Isso é necessário porque alguns clãs possuem duas ou mais linhas do mesmo jogador, com resultados diferentes.
Com o total de troféus, podemos concatená-lo com os dados da porcentagem de vitórias, selecionar os 10 melhores clãs com [20% ou mais de vitórias] e [100.000 troféus ou mais] e ver os decks mais utilizados por eles.

In [17]:
# União entre melhores clãs e maiores troféus

#Abrindo as informações de troféus e selecionando os melhores
CRMf['Trophy'] = [int(trophy) for trophy in CRMf['Trophy']]
MediaTrofeus = CRMf.groupby('Clan')['Trophy'].mean()
PlayersCla = CRMf.groupby('Clan')['Player'].nunique()
ClasMaisTrofeus = (PlayersCla * MediaTrofeus)

# Top Clãs e seus decks
TopClas = pd.concat([PercVitoriaCla, ClasMaisTrofeus], axis=1, join='inner')
TopClas.columns = ['Percentual', 'Trophy']
Top10Clans = TopClas[(TopClas['Percentual']>20.0) & (TopClas['Trophy']>100000.0)].sort_values(by=['Trophy', 'Percentual'], ascending=False).head(10)

TopClansDecks = CRMf[(CRMf['Clan']==Top10Clans.index[0]) | (CRMf['Clan']== Top10Clans.index[1]) | (CRMf['Clan']== Top10Clans.index[2]) | (CRMf['Clan']== Top10Clans.index[3]) | (CRMf['Clan']== Top10Clans.index[4]) | (CRMf['Clan']== Top10Clans.index[5]) | (CRMf['Clan']== Top10Clans.index[6]) | (CRMf['Clan']== Top10Clans.index[7]) | (CRMf['Clan']== Top10Clans.index[8]) | (CRMf['Clan']== Top10Clans.index[9])]
print('*'*30)
print('Decks dos Melhores Clãs')
print('*'*30)
TopClansDecks[['Troop 1', 'Troop 2', 'Troop 3', 'Troop 4', 'Troop 5', 'Troop 6', 'Troop 7', 'Troop 8', 'Result']].groupby(['Troop 1', 'Troop 2', 'Troop 3', 'Troop 4', 'Troop 5', 'Troop 6', 'Troop 7', 'Troop 8']).count().sort_values(by='Result',ascending=False).head(10)

******************************
Decks dos Melhores Clãs
******************************


,,,,,,,,Result
Troop 1,Troop 2,Troop 3,Troop 4,Troop 5,Troop 6,Troop 7,Troop 8,
Giant,Graveyard,Knight,Arrows,Goblin Gang,Minions,Archers,Zap,86
Skeletons,Archers,Tombstone,Poison,Baby Dragon,P.E.K.K.A,The Log,Graveyard,79
Knight,Ice Spirit,Miner,The Log,Three Musketeers,Skeletons,Minion Horde,Elixir Collector,74
Goblins,The Log,Ice Golem,Battle Ram,Poison,Bandit,Night Witch,Inferno Dragon,71
Knight,Minion Horde,Ice Spirit,Zap,Three Musketeers,Battle Ram,Heal,Elixir Collector,67
Golem,Baby Dragon,Night Witch,Elixir Collector,Dark Prince,Zap,Tornado,Inferno Dragon,65
Giant,Mega Minion,Night Witch,Electro Wizard,Miner,Fireball,Zap,The Log,64
P.E.K.K.A,Baby Dragon,Goblin Gang,Fireball,Battle Ram,Miner,Zap,Archers,57
Fire Spirits,Poison,Guards,Electro Wizard,Graveyard,Night Witch,The Log,Ice Golem,54


Assim, se você é um iniciante no Clash Royale ou está procurando alguma sugestão de baralho, aqui estão os baralhos mais comuns dos clãs com maior vitória, mesmo quando eles têm troféus mais altos.

A seguir: Outras sugestões > Decks bons de Defesa; Decks bons de Ataque:

In [6]:
# Players bons de defesa (e seus decks)

CRMf['Crowns Won'] = [int(crowns) for crowns in CRMf['Crowns Won']]
CRMf['Crowns Lost'] = [int(crowns) for crowns in CRMf['Crowns Lost']]
PlayersDef = CRMf[(CRMf['Result']==1) & (CRMf['Crowns Lost']==0)]
Top10PlayersD = pd.DataFrame(PlayersDef.groupby('Player')['Result'].sum().sort_values(ascending=False).head(11))

PlayersDefDecks = PlayersDef[(PlayersDef['Player']==Top10PlayersD.index[1]) | (PlayersDef['Player']==Top10PlayersD.index[2]) | (PlayersDef['Player']==Top10PlayersD.index[3]) | (PlayersDef['Player']==Top10PlayersD.index[4]) | (PlayersDef['Player']==Top10PlayersD.index[5]) | (PlayersDef['Player']==Top10PlayersD.index[6]) | (PlayersDef['Player']==Top10PlayersD.index[7]) | (PlayersDef['Player']==Top10PlayersD.index[8]) | (PlayersDef['Player']==Top10PlayersD.index[9]) | (PlayersDef['Player']==Top10PlayersD.index[10])]
print('*'*30)
print('Melhores Decks de Defesa')
print('*'*30)
PlayersDefDecks[['Troop 1', 'Troop 2', 'Troop 3', 'Troop 4', 'Troop 5', 'Troop 6', 'Troop 7', 'Troop 8', 'Result']].groupby(['Troop 1', 'Troop 2', 'Troop 3', 'Troop 4', 'Troop 5', 'Troop 6', 'Troop 7', 'Troop 8']).count().sort_values(by='Result',ascending=False).head(10)

******************************
Melhores Decks de Defesa
******************************


Result
Troop 1        Troop 2      Troop 3     Troop 4     Troop 5          Troop 6        Troop 7       Troop 8                 
Knight         Minion Horde Ice Spirit  Zap         Three Musketeers Battle Ram     Heal          Elixir Collector      29
Tombstone      Rocket       Knight      The Log     Princess         Dart Goblin    Goblin Barrel Goblin Gang           29
Ice Spirit     Fireball     Archers     Goblins     Knight           Hog Rider      Zap           Cannon                24
Electro Wizard Goblins      Baby Dragon Graveyard   Bowler           Poison         Tornado       Knight                18
Tornado        Hog Rider    Executioner P.E.K.K.A   Knight           Lightning      The Log       Dart Goblin           14
Hog Rider      Mega Minion  Ice Golem   Fireball    Tombstone        Goblins        Princess      The Log               11
Poison         Battle Ram   Night Witch P.E.K.K.A   Elixir Collector Bandit         The Log       Baby Dragon           11
               Night Witch  Ice Spirit  Ice Golem   Graveyard        Electro Wizard The Log       Elixir Collector      10
Hog Rider      Mega Minion  Tombstone   Poison      Fire Spirits     Knight         Ice Wizard    The Log                9
Furnace        Knight       Mega Minion Royal Giant Archers          Fireball       The Log       Ice Spirit             8

In [7]:
# Players bons de ataque (e seus decks)

PlayersAtk = CRMf[(CRMf['Result']==1) & (CRMf['Crowns Won']==3)]
Top10PlayersA = PlayersAtk.groupby('Player')['Result'].sum().sort_values(ascending=False).head(11)
print('*'*30)
print('Melhores Decks de Ataque')
print('*'*30)
PlayersAtkDecks = PlayersAtk[(PlayersAtk['Player']==Top10PlayersA.index[1]) | (PlayersAtk['Player']==Top10PlayersA.index[2]) | (PlayersAtk['Player']==Top10PlayersA.index[3]) | (PlayersAtk['Player']==Top10PlayersA.index[4]) | (PlayersAtk['Player']==Top10PlayersA.index[5]) | (PlayersAtk['Player']==Top10PlayersA.index[6]) | (PlayersAtk['Player']==Top10PlayersA.index[7]) | (PlayersAtk['Player']==Top10PlayersA.index[8]) | (PlayersAtk['Player']==Top10PlayersA.index[9]) | (PlayersAtk['Player']==Top10PlayersA.index[10])]
PlayersAtkDecks[['Troop 1', 'Troop 2', 'Troop 3', 'Troop 4', 'Troop 5', 'Troop 6', 'Troop 7', 'Troop 8', 'Result']].groupby(['Troop 1', 'Troop 2', 'Troop 3', 'Troop 4', 'Troop 5', 'Troop 6', 'Troop 7', 'Troop 8']).count().sort_values(by='Result',ascending=False).head(10)

******************************
Melhores Decks de Ataque
******************************


Result
Troop 1      Troop 2          Troop 3      Troop 4          Troop 5          Troop 6          Troop 7          Troop 8                 
Knight       Heal             Ice Golem    Elixir Collector Minion Horde     Hog Rider        Mirror           Three Musketeers      25
Battle Ram   Night Witch      Knight       Three Musketeers Elixir Collector Zap              Skeletons        Ice Spirit            19
Golem        Baby Dragon      Night Witch  Elixir Collector Dark Prince      Zap              Tornado          Inferno Dragon        18
Minion Horde Battle Ram       Zap          Goblins          Ice Golem        Elixir Collector Three Musketeers Heal                  16
Royal Giant  Elite Barbarians Hog Rider    Heal             Electro Wizard   Arrows           Clone            Minion Horde          15
Night Witch  Guards           Golem        Tornado          Mega Minion      Elixir Collector Baby Dragon      Lightning             12
Minion Horde Elixir Collector Zap          Three Musketeers Heal             Battle Ram       Goblins          Ice Golem             12
P.E.K.K.A    Night Witch      Minion Horde Battle Ram       Three Musketeers Elixir Collector Zap              Heal                  11
             Baby Dragon      Bandit       Poison           Battle Ram       Elixir Collector The Log          Night Witch            9
Zap          Battle Ram       Minions      Ice Golem        Three Musketeers Elixir Collector Mirror           Bandit                 9